In [ ]:
import time
import gc
import pickle
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Reshape, concatenate, Concatenate, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D
from tensorflow.keras.layers import Input, Activation, BatchNormalization, Dropout, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping,TensorBoard,LearningRateScheduler
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, jaccard_score
from keras.callbacks import CSVLogger
from keras.optimizers import AdamW

from sklearn.model_selection import train_test_split
from load_data import load_data
from Evaluation_Metrics import metrics as m
from Augmentation import augment
from Train import train
from Model import deeplabv1, deeplabv3, doubleunet, unet

In [ ]:
img_size = 352
X, Y = load_data(img_size, img_size, -1) #Resize (352,352) for Kvasir-SEG & CVC-ColonDB while (256,256) for CVC-ClinicDB

In [ ]:
test_size = 0.2
validation_size = 0.5

#Split the the data into train:validation:test with 8:1:1 Ratio
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=validation_size, random_state=0)

# Layer 1

In [ ]:
#If Output is float32 use dice_coeff2 and IoU2 else use dice_coeff and IoU

In [ ]:
epoch = 600
lr = 0.0001
dataset_type=""
model_type = ""
model_path_1 = '' + dataset_type + '_' + model_type

In [ ]:
#Compile DeepLabV3 Model
model_type_1 = deeplabv3.DeepLabV3(modelType="ResNet101",shape=(352,352,3))
optimizer = AdamW(learning_rate = lr, weight_decay = 0.004)
loss = 'binary_crossentropy'
model_type_1.compile(optimizer=optimizer, loss=loss,
                     metrics=['accuracy',m.f1_score,m.dice_coeff2, m.dice_loss, m.total_loss, m.IoU, m.zero_IoU, tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [ ]:
train.train_model(model=model_type_1, EPOCHS=epoch, X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test, learning_rate=lr, model_path=model_path_1)

In [ ]:
model_type = ""
model_path_2 = '' + dataset_type + '_' + model_type

In [ ]:
model_type_2 = doubleunet.double_unet(352,352,3)
optimizer = AdamW(
    learning_rate = 0.00001, weight_decay = 0.004)
loss = 'binary_crossentropy'
model_type_2.compile(optimizer=optimizer, loss=loss,
                     metrics=['accuracy',m.f1_score2,m.dice_coeff, m.dice_loss, m.total_loss, m.IoU, m.zero_IoU, tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [ ]:
train.train_model(model=model_type_2, EPOCHS=epoch, X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test, learning_rate=lr, model_path=model_path_2)

# Evaluation for Layer 1


In [ ]:
print("Loading the models")

model_type_1 = tf.keras.models.load_model(model_path_1, 
                                          custom_objects={'f1_score': m.f1_score,
    'dice_coeff': m.dice_coeff,
    'dice_loss': m.dice_loss,
    'total_loss': m.total_loss,
    'IoU': m.IoU,
    'zero_IoU': m.zero_IoU,
    'dice_metric_loss': m.dice_metric_loss})
model_type_1_preds_tr = model_type_1.predict(X_train, batch_size=4)
model_type_1_preds_val = model_type_1.predict(X_val, batch_size=4)
model_type_1_preds_t = model_type_1.predict(X_test, batch_size=4)

model_type_2 = tf.keras.models.load_model(model_path_2, 
                                          custom_objects={'f1_score': m.f1_score2,
    'dice_coeff': m.dice_coeff,
    'dice_loss': m.dice_loss,
    'total_loss': m.total_loss,
    'IoU': m.IoU,
    'zero_IoU': m.zero_IoU,
    'dice_metric_loss': m.dice_metric_loss})
model_type_2_preds_tr = model_type_2.predict(X_train, batch_size=4)
model_type_2_preds_val = model_type_2.predict(X_val, batch_size=4)
model_type_2_preds_t = model_type_2.predict(X_test, batch_size=4)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, jaccard_score

ground_truth_flat = y_test.flatten()
predictions_flat = np.round(model_type_1_preds_t).flatten()

# Calculate metrics
accuracy = accuracy_score(ground_truth_flat, predictions_flat)
precision = precision_score(ground_truth_flat, predictions_flat)
recall = recall_score(ground_truth_flat, predictions_flat)
f1 = f1_score(ground_truth_flat, predictions_flat)
jaccard = jaccard_score(ground_truth_flat, predictions_flat)

# Print or log the evaluation metrics
print("Model_Type_1: ")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")
print(f"IoU: {jaccard}")

predictions_flat = np.round(model_type_2_preds_t).flatten()

accuracy = accuracy_score(ground_truth_flat, predictions_flat)
precision = precision_score(ground_truth_flat, predictions_flat)
recall = recall_score(ground_truth_flat, predictions_flat)
f1 = f1_score(ground_truth_flat, predictions_flat)
jaccard = jaccard_score(ground_truth_flat, predictions_flat)

# Print or log the evaluation metrics
print("Model_Type_2: ")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")
print(f"IoU: {jaccard}")

## New inputs for Layer 2

In [ ]:
#Combine Model_type_1 and Model_type_2 Train predictions using averaging method and concatenate it with the training set.
average_preds_tr = (model_type_1_preds_tr + model_type_2_preds_tr) / 2.0
train_with_predictions = np.concatenate([X_train, average_preds_tr], axis=-1)

#Combine Model_type_1 and Model_type_2 Validation Predictions using averaging method and concatenate it with the validation set.
average_preds_val = (model_type_1_preds_val + model_type_2_preds_val) / 2.0
val_with_predictions = np.concatenate([X_val, average_preds_val], axis=-1)

#Combine Model_type_1 and Model_type_2 Predictions using averaging method and concatenate it with the validation set.
average_preds_t = (model_type_1_preds_t + model_type_2_preds_t) / 2.0
test_with_predictions = np.concatenate([X_test, average_preds_t], axis=-1)

# Layer 2

In [ ]:
from datetime import datetime

lr2 = 0.00001
log_dir = './logs/fit' + datetime.now().strftime('%d.%m.%Y--%H-%M-%S')
model_path_3 = ''
checkpoint = ModelCheckpoint(model_path_3, monitor='val_dice_metric_loss', verbose = 1, save_best_only=True,
                            mode='min', save_freq='epoch')
early = EarlyStopping(monitor='val_dice_metric_loss', min_delta=0, patience = 25, verbose = 1, mode='min')
board = TensorBoard(log_dir=log_dir,histogram_freq = 1)
tensorboard_callback = [checkpoint,early,board]

In [ ]:
optimizer = AdamW(learning_rate = lr2, weight_decay = 0.004)
loss = 'binary_crossentropy'
model_type_3 = unet.unet(input_shape=(352,352,4))
model_type_3.compile(optimizer=optimizer, loss=loss, 
                    metrics=['accuracy',m.f1_score2,m.dice_metric_loss,m.dice_coeff,m.dice_loss, m.total_loss, m.IoU, m.zero_IoU, tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [ ]:
model_type_3.fit(train_with_predictions, y_train,
                  batch_size=8,epochs=400,
                  validation_data=(val_with_predictions, y_val),
                  callbacks=[tensorboard_callback])

In [ ]:
from datetime import datetime

log_dir = './logs/fit' + datetime.now().strftime('%d.%m.%Y--%H-%M-%S')
model_path_4 = ''
checkpoint = ModelCheckpoint(model_path_4, monitor='val_dice_metric_loss', verbose = 1, save_best_only=True,
                            mode='min', save_freq='epoch')
early = EarlyStopping(monitor='val_dice_metric_loss', min_delta=0, patience = 25, verbose = 1, mode='min')
board = TensorBoard(log_dir=log_dir,histogram_freq = 1)
tensorboard_callback = [checkpoint,early,board]

In [ ]:
optimizer = AdamW(learning_rate = lr2, weight_decay = 0.004)
loss = 'binary_crossentropy'
model_type_4 = deeplabv1.deeplab(input_shape=(352,352,4))
model_type_4.compile(optimizer=optimizer, loss=loss, 
                       metrics=['accuracy',m.f1_score2,m.dice_metric_loss,m.dice_coeff,m.dice_loss, m.total_loss, m.IoU, m.zero_IoU, tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [ ]:
model_type_4.fit(train_with_predictions, y_train,
                  batch_size=8,epochs=400,
                  validation_data=(val_with_predictions, y_val),
                  callbacks=[tensorboard_callback])

## Layer 2 Evaluation

In [ ]:
model_type_3 = tf.keras.models.load_model(model_path_3, 
                                          custom_objects={'f1_score': m.f1_score2,
        'dice_coeff': m.dice_coeff,
        'dice_loss': m.dice_loss,
        'total_loss': m.total_loss,
        'IoU': m.IoU,
        'zero_IoU': m.zero_IoU,
        'dice_metric_loss':m.dice_metric_loss})
model_type_3_t = model_type_3.predict(test_with_predictions,batch_size=4)

model_type_4 = tf.keras.models.load_model(model_path_4, 
                                          custom_objects={'f1_score': m.f1_score,
        'dice_coeff': m.dice_coeff,
        'dice_loss': m.dice_loss,
        'total_loss': m.total_loss,
        'IoU': m.IoU,
        'zero_IoU': m.zero_IoU,
        'dice_metric_loss':m.dice_metric_loss})
model_type_4_t = model_type_4.predict(test_with_predictions,batch_size=4)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, jaccard_score

ground_truth_flat = y_test.flatten()
predictions_flat = np.round(model_type_3_t).flatten()

accuracy = accuracy_score(ground_truth_flat, predictions_flat)
precision = precision_score(ground_truth_flat, predictions_flat)
recall = recall_score(ground_truth_flat, predictions_flat)
f1 = f1_score(ground_truth_flat, predictions_flat)
jaccard = jaccard_score(ground_truth_flat, predictions_flat)

print("Model_Type_3: ")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")
print(f"IoU: {jaccard}")

predictions_flat = np.round(model_type_4_t).flatten()

accuracy = accuracy_score(ground_truth_flat, predictions_flat)
precision = precision_score(ground_truth_flat, predictions_flat)
recall = recall_score(ground_truth_flat, predictions_flat)
f1 = f1_score(ground_truth_flat, predictions_flat)
jaccard = jaccard_score(ground_truth_flat, predictions_flat)

print("Model_Type_4: ")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")
print(f"IoU: {jaccard}")

# Final Output Prediction

In [ ]:
#Combine Model_Type_3 and Model_Type_4 preds by using the averaging method and we get the final prediction.
average2_preds_test = (model_type_3_t + model_type_4_t) / 2.0
average2_preds_t = np.round(average2_preds_test).flatten()
y_test_flat = y_test.flatten()

In [ ]:
avg_ensemble_accuracy = accuracy_score(y_test_flat, average2_preds_t)
avg_ensemble_f1 = f1_score(y_test_flat, average2_preds_t)
avg_ensemble_precision = precision_score(y_test_flat, average2_preds_t)
avg_ensemble_recall = recall_score(y_test_flat, average2_preds_t)
avg_ensemble_iou = jaccard_score(y_test_flat, average2_preds_t)

# Display ensemble metrics
print("Averaged Ensemble Accuracy:", avg_ensemble_accuracy)
print("Averaged Ensemble F1 Score:", avg_ensemble_f1)
print("Averaged Ensemble Precision:", avg_ensemble_precision)
print("Averaged Ensemble Recall:", avg_ensemble_recall)
print("Averaged Ensemble IoU:", avg_ensemble_iou)

# Visualization of Models from layer 1, layer 2, and final output

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

num_samples_to_visualize = 20
random_indices = np.random.choice(range(len(X_test)), num_samples_to_visualize, replace=False)

# Visualize each selected sample
for sample_index in random_indices:
    # Load the original image and corresponding mask
    original_image = X_test[sample_index]
    actual_mask = y_test[sample_index]

    # Get predictions for the chosen sample
    doubleunet_prediction = np.round(model_type_1_preds_t[sample_index, ..., 0])
    deeplabv3p_prediction = np.round(model_type_2_preds_t[sample_index, ..., 0])
    avg_prediction = np.round(average_preds_t[sample_index])
    unet2_prediction = np.round(model_type_3_t[sample_index, ..., 0])
    deeplab2_prediction = np.round(model_type_4_t[sample_index, ..., 0])
    avg2_prediction = np.round(average2_preds_test[sample_index])
    # Plot the images and predictions
    plt.figure(figsize=(20, 20))

    plt.subplot(1, 11, 1)
    plt.imshow(original_image)
    plt.title("Original Image")

    plt.subplot(1, 11, 2)
    plt.imshow(original_image[:, :, 0], cmap='Reds')
    plt.title("Red Channel")

    plt.subplot(1, 11, 3)
    plt.imshow(original_image[:, :, 1], cmap='Greens')
    plt.title("Green Channel")

    plt.subplot(1, 11, 4)
    plt.imshow(original_image[:, :, 2], cmap='Blues')
    plt.title("Blue Channel")

    plt.subplot(1, 11, 5)
    plt.imshow(actual_mask, cmap='gray')
    plt.title("Actual Mask")

    plt.subplot(1, 11, 6)
    plt.imshow(doubleunet_prediction, cmap='gray')
    plt.title("DoubleU-Net")

    plt.subplot(1, 11, 7)
    plt.imshow(deeplabv3p_prediction, cmap='gray')
    plt.title("Deeplab V3 Plus")

    plt.subplot(1, 11, 8)
    plt.imshow(avg_prediction, cmap='gray')
    plt.title("Avg Layer 1")

    plt.subplot(1, 11, 9)
    plt.imshow(unet2_prediction, cmap='gray')
    plt.title("U-Net 2")

    plt.subplot(1, 11, 10)
    plt.imshow(deeplab2_prediction, cmap='gray')
    plt.title("Deeplab 2")

    plt.subplot(1, 11, 11)
    plt.imshow(avg2_prediction, cmap='gray')
    plt.title("Avg Layer 2")


    plt.tight_layout()
    plt.show()